In [1]:
import requests
import json
import sys
import pandas as pd
import numpy as np

from  keys  import  client_id, api_key
#import Yelp_Helper_Functions as yf

In [2]:
url        = 'https://api.yelp.com/v3/businesses/search'
headers    = {
             'Authorization': 'Bearer {}'.format(api_key),
             }
term       = 'bars'
location   = 'Boston'
categories = 'restaurants'
url_params = {
                "term"       : term.replace(' ', '+'),
                "location"   : location.replace(' ', '+'),
                "categories" : categories,
                "limit"      : 50,
                "offset"     : 0
              }
response   = requests.get(url, headers=headers, params=url_params)

In [3]:
def yelp_biz_call(url_params, api_key):
    response = requests.get(url, headers=headers, params=url_params)
    return response.json()['businesses']

In [4]:
def parse_biz_data(list_of_data):
    businesses = []
    for business in list_of_data:
        if 'price' not in business.keys():
            business['price'] = np.nan
        if 'delivery' in business['transactions']:
            business['delivery'] = True
        else:
            business['delivery'] = False
        biz_tuple = (business['id'],
                     business['name'],
                     business['location']['zip_code'],
                     business['rating'],
                     business['review_count'],
                     business['categories'][0]['alias'],
                     business['delivery'],
                     business['distance'],
                     business['price']
                    )
        businesses.append(biz_tuple)
    return businesses

In [5]:
def df_save_bars_denver(yb_data, parse_data):
    yb_data = pd.concat([yb_data, parse_data], axis='rows')  
    yb_data.to_csv("Bars_Boston.csv",
                   mode='a', index=False, header=False)
    return

In [6]:
cur     = 0
num     = 1000
yb_data = []
yb_data = pd.DataFrame(yb_data)
yb_data.to_csv("Bars_Boston.csv", 
               index_label=['id', 'Name', 'Zip Code', 
                            'Buisness Rating', 'Review Count',
                            'Categories', 'Delivery', 
                            'Distance', 'Price'])
while cur < num :
    url_params['offset'] = cur
    results     = yelp_call(url_params, api_key)
    parsed_data = parse_data(results)
    parsed_data = pd.DataFrame(parsed_data)
    df_save(yb_data, parsed_data)
    cur += 50